In [2]:
import sys
from pathlib import Path
sys.path.append(str(Path().resolve().parent))

In [4]:
from utils.notes_processing import generate_spectrogram
from models.dataset import GoodSoundsDatabase, GoodSoundsDataset

In [5]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

# Initial CNN test with just trumpet recordings

We are first going to train a small-scale CNN on soley trumpet note spectrograms to see what kind of results we get.

In [6]:
#Get dataframe for trumpet sounds
db = GoodSoundsDatabase("/Users/dhanush/documents/musaic/good-sounds")
trumpet_df = db.get_sounds_by_instrument("trumpet")
trumpet_df.head()

INFO:models.dataset:Connected to database: /Users/dhanush/documents/musaic/good-sounds/database.sqlite


,id,instrument,note,octave,dynamics,recorded_at,location,player,bow_velocity,bridge_position,...,decay,sustain,release,offset,reference,klass,comments,semitone,pitch_reference,file_path
0,1020,trumpet,A#,5,mf,2014-04-24 11:30:00.000000,upf studio,ramon,None,None,...,NaN,None,NaN,NaN,1,good-sound,,70,440.0,/Users/dhanush/documents/musaic/good-sounds/so...
1,1021,trumpet,A,5,mf,2014-04-24 11:30:00.000000,upf studio,ramon,None,None,...,9195.0,None,199445.0,213500.0,1,good-sound,,69,440.0,/Users/dhanush/documents/musaic/good-sounds/so...
2,1022,trumpet,G#,5,mf,2014-04-24 11:30:00.000000,upf studio,ramon,None,None,...,12271.0,None,169546.0,184641.0,1,good-sound,,68,440.0,/Users/dhanush/documents/musaic/good-sounds/so...
3,1023,trumpet,G,5,mf,2014-04-24 11:30:00.000000,upf studio,ramon,None,None,...,NaN,None,NaN,NaN,1,good-sound,,67,440.0,/Users/dhanush/documents/musaic/good-sounds/so...
4,1024,trumpet,F#,5,mf,2014-04-24 11:30:00.000000,upf studio,ramon,None,None,...,22495.0,None,187446.0,201000.0,1,good-sound,,66,440.0,/Users/dhanush/documents/musaic/good-sounds/so...


In [7]:
#Analyze label distribution

print(f"\n Label Distribution:")
print("-" * 40)

label_counts = trumpet_df['klass'].value_counts()
total = len(trumpet_df)

for label, count in label_counts.items():
    percentage = (count / total) * 100
    print(f"{label}: {count} samples ({percentage:.1f}%)")


 Label Distribution:
----------------------------------------
good-sound: 90 samples (14.2%)
bad-dynamics-stability-errors: 62 samples (9.8%)
bad-timbre-stability-errors: 53 samples (8.4%)
bad-pitch: 45 samples (7.1%)
good-attack: 32 samples (5.1%)
bad-attack: 32 samples (5.1%)
bad-attack-air: 29 samples (4.6%)
too-much-air: 29 samples (4.6%)
bad-attack-pitch-up: 29 samples (4.6%)
good-attack-no-picat: 29 samples (4.6%)
bad-pitch-stability-errors: 29 samples (4.6%)
bad-pitch-stability-bend: 29 samples (4.6%)
bad-dynamics-stability-decrescendo: 29 samples (4.6%)
bad-dynamics-stability-crescendo: 27 samples (4.3%)
bad-richness: 26 samples (4.1%)
bad-dynamics-stability-tremolo: 25 samples (4.0%)
bad-attack-pitch-down: 24 samples (3.8%)
good-attack-soft: 13 samples (2.1%)


In [8]:
#Create training, validation, and testing sets
test_size=0.2
val_size=0.2
random_state = 42
stratify = True

# First split: separate test set
stratify_col = trumpet_df['klass'] if stratify else None

train_val_df, test_df = train_test_split(
    trumpet_df, 
    test_size=test_size, 
    random_state=random_state,
    stratify=stratify_col
)

# Second split: separate validation from training
stratify_col_remaining = train_val_df['klass'] if stratify else None

train_df, val_df = train_test_split(
    train_val_df,
    test_size=val_size,
    random_state=random_state,
    stratify=stratify_col_remaining
)

print(f"Data split summary:")
print(f"Total samples: {len(trumpet_df)}")
print(f"Training: {len(train_df)} ({len(train_df)/len(trumpet_df)*100:.1f}%)")
print(f"Validation: {len(val_df)} ({len(val_df)/len(trumpet_df)*100:.1f}%)")
print(f"Testing: {len(test_df)} ({len(test_df)/len(trumpet_df)*100:.1f}%)")

Data split summary:
Total samples: 632
Training: 404 (63.9%)
Validation: 101 (16.0%)
Testing: 127 (20.1%)


In [9]:
#Create label encoder

label_encoder = LabelEncoder()
label_encoder.fit(train_df['klass'])

print(f"\nLabel Encoding:")
print("-" * 20)
for i, label in enumerate(label_encoder.classes_):
    print(f"{i}: {label}")


Label Encoding:
--------------------
0: bad-attack
1: bad-attack-air
2: bad-attack-pitch-down
3: bad-attack-pitch-up
4: bad-dynamics-stability-crescendo
5: bad-dynamics-stability-decrescendo
6: bad-dynamics-stability-errors
7: bad-dynamics-stability-tremolo
8: bad-pitch
9: bad-pitch-stability-bend
10: bad-pitch-stability-errors
11: bad-richness
12: bad-timbre-stability-errors
13: good-attack
14: good-attack-no-picat
15: good-attack-soft
16: good-sound
17: too-much-air


In [25]:
spectrogram_function = generate_spectrogram
cache_spectrograms = False
cache_dir = None
batch_size=1
num_workers=4

# Create datasets
train_dataset = GoodSoundsDataset(
    train_df, spectrogram_function, label_encoder,
    cache_spectrograms=cache_spectrograms, cache_dir=cache_dir
)

val_dataset = GoodSoundsDataset(
    val_df, spectrogram_function, label_encoder,
    cache_spectrograms=cache_spectrograms, cache_dir=cache_dir
)

test_dataset = GoodSoundsDataset(
    test_df, spectrogram_function, label_encoder,
    cache_spectrograms=cache_spectrograms, cache_dir=cache_dir
)

# Create data loaders
train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, #collate_fn=lambda x: x[0]
)

val_loader = DataLoader(
    val_dataset, batch_size=batch_size, shuffle=False, #collate_fn=lambda x: x[0]
)

test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, #collate_fn=lambda x: x[0]
)

In [26]:
#Test that data loading works correctly
data_loader = train_loader
num_samples = 3
    
print(f"\nTesting data loading with {num_samples} samples...")

for i, batch in enumerate(data_loader):
    if i >= num_samples:
        break
        
    spectrograms = batch['spectrogram']
    labels = batch['label']
    
    print(f"Batch {i+1}:")
    print(f"  Spectrogram shape: {spectrograms.shape}")
    print(f"  Labels shape: {labels.shape}")
    print(f"  Label values: {labels.numpy()}")
    print(f"  Spectrogram range: [{spectrograms.min():.3f}, {spectrograms.max():.3f}]")


Testing data loading with 3 samples...
Batch 1:
  Spectrogram shape: torch.Size([1, 887, 1024])
  Labels shape: torch.Size([1])
  Label values: [12]
  Spectrogram range: [0.000, 64.754]


/Users/dhanush/miniconda3/envs/musaic_env/lib/python3.9/site-packages/madmom/io/audio.py:643: WavFileWarning: Chunk (non-data) not understood, skipping it.
  file_sample_rate, signal = wavfile.read(filename, mmap=True)
/Users/dhanush/miniconda3/envs/musaic_env/lib/python3.9/site-packages/madmom/io/audio.py:643: WavFileWarning: Chunk (non-data) not understood, skipping it.
  file_sample_rate, signal = wavfile.read(filename, mmap=True)
/Users/dhanush/miniconda3/envs/musaic_env/lib/python3.9/site-packages/madmom/io/audio.py:643: WavFileWarning: Chunk (non-data) not understood, skipping it.
  file_sample_rate, signal = wavfile.read(filename, mmap=True)


Batch 2:
  Spectrogram shape: torch.Size([1, 355, 1024])
  Labels shape: torch.Size([1])
  Label values: [0]
  Spectrogram range: [0.000, 72.250]
Batch 3:
  Spectrogram shape: torch.Size([1, 857, 1024])
  Labels shape: torch.Size([1])
  Label values: [12]
  Spectrogram range: [0.000, 55.849]


/Users/dhanush/miniconda3/envs/musaic_env/lib/python3.9/site-packages/madmom/io/audio.py:643: WavFileWarning: Chunk (non-data) not understood, skipping it.
  file_sample_rate, signal = wavfile.read(filename, mmap=True)


In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

In [15]:
class VariableLengthCNN(nn.Module):
    """CNN that handles variable-length spectrograms using global pooling"""
    
    def __init__(self, num_classes=5, num_freq_bins=128):
        super().__init__()
        
        # Convolutional layers that preserve time dimension
        self.conv_layers = nn.Sequential(
            # First conv block
            nn.Conv2d(1, 32, kernel_size=(7, 3), padding=(3, 1)),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),  # Only pool frequency dimension
            
            # Second conv block
            nn.Conv2d(32, 64, kernel_size=(5, 3), padding=(2, 1)),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),
            
            # Third conv block
            nn.Conv2d(64, 128, kernel_size=(3, 3), padding=(1, 1)),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d((2, 1)),
            
            # Fourth conv block
            nn.Conv2d(128, 256, kernel_size=(3, 3), padding=(1, 1)),
            nn.BatchNorm2d(256),
            nn.ReLU(),
        )
        
        # Calculate frequency dimension after pooling
        self.freq_dim_after_conv = num_freq_bins // (2 * 2 * 2)  # 3 pooling layers
        
        # Global pooling across time dimension
        self.global_avg_pool = nn.AdaptiveAvgPool2d((self.freq_dim_after_conv, 1))
        self.global_max_pool = nn.AdaptiveMaxPool2d((self.freq_dim_after_conv, 1))
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * self.freq_dim_after_conv * 2, 512),  # *2 for avg+max pooling
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )
    
    def forward(self, x):
        # x shape: (batch, 1, freq, time) - time can vary per sample
        
        # Extract features with convolutions
        features = self.conv_layers(x)  # (batch, 256, freq', time)
        
        # Global pooling to handle variable time dimension
        avg_pooled = self.global_avg_pool(features)  # (batch, 256, freq', 1)
        max_pooled = self.global_max_pool(features)  # (batch, 256, freq', 1)
        
        # Combine avg and max pooling
        combined = torch.cat([avg_pooled, max_pooled], dim=1)  # (batch, 512, freq', 1)
        
        # Flatten for classifier
        flattened = combined.view(combined.size(0), -1)
        
        # Classify
        output = self.classifier(flattened)
        
        return output

In [30]:
# Training function for variable length inputs
def train_variable_length_model(model, train_loader, val_loader, num_epochs=10):
    """Training loop that handles variable length inputs"""
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        
        for i, batch in enumerate(train_loader):
            spectrograms = batch['spectrogram']
            labels = batch['label'].to(device)
            
            optimizer.zero_grad()
            
            # Process each spectrogram individually if they're in a list
            if isinstance(spectrograms, list):
                batch_outputs = []
                for spec in spectrograms:
                    if spec.dim() == 2:  # (freq, time)
                        spec = spec.unsqueeze(0).unsqueeze(0)  # Add batch and channel dims
                    elif spec.dim() == 3:  # (channels, freq, time) or (batch, freq, time)
                        if spec.shape[0] == 1:  # Assume (channels=1, freq, time)
                            spec = spec.unsqueeze(0)  # Add batch dim -> (1, 1, freq, time)
                        else:  # Assume (batch=1, freq, time) - add channel dim
                            spec = spec.unsqueeze(1)  # -> (1, 1, freq, time)
                    spec = spec.to(device)
                    output = model(spec)
                    batch_outputs.append(output)
                outputs = torch.cat(batch_outputs, dim=0)
            else:
                # Handle batched spectrograms
                if spectrograms.dim() == 3:  # (batch, freq, time) - missing channel dim
                    spectrograms = spectrograms.unsqueeze(1)  # Add channel -> (batch, 1, freq, time)
                elif spectrograms.dim() == 2:  # Single sample (freq, time)
                    spectrograms = spectrograms.unsqueeze(0).unsqueeze(0)  # -> (1, 1, freq, time)
                
                spectrograms = spectrograms.to(device)
                outputs = model(spectrograms)
            
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, predicted = outputs.max(1)
            train_total += labels.size(0)
            train_correct += predicted.eq(labels).sum().item()
            print(F"{i} out of {len(train_loader)} training complete")
        
        # Validation
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        
        with torch.no_grad():
            for i, batch in enumerate(val_loader):
                spectrograms = batch['spectrogram']
                labels = batch['label'].to(device)
                
                if isinstance(spectrograms, list):
                    batch_outputs = []
                    for spec in spectrograms:
                        # Same dimension handling as training
                        if spec.dim() == 2:
                            spec = spec.unsqueeze(0).unsqueeze(0)
                        elif spec.dim() == 3:
                            if spec.shape[0] == 1:
                                spec = spec.unsqueeze(0)
                            else:
                                spec = spec.unsqueeze(1)
                        
                        spec = spec.to(device)
                        output = model(spec)
                        batch_outputs.append(output)
                    outputs = torch.cat(batch_outputs, dim=0)
                else:
                    if spectrograms.dim() == 3:
                        spectrograms = spectrograms.unsqueeze(1)
                    elif spectrograms.dim() == 2:
                        spectrograms = spectrograms.unsqueeze(0).unsqueeze(0)
                    
                    spectrograms = spectrograms.to(device)
                    outputs = model(spectrograms)
                
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_total += labels.size(0)
                val_correct += predicted.eq(labels).sum().item()
                print(F"{i} out of {len(val_loader)} validation complete")
        
        print(f'Epoch {epoch+1}/{num_epochs}:')
        print(f'  Train Loss: {train_loss/len(train_loader):.4f}, Acc: {100.*train_correct/train_total:.2f}%')
        print(f'  Val Loss: {val_loss/len(val_loader):.4f}, Acc: {100.*val_correct/val_total:.2f}%')

In [31]:
model = VariableLengthCNN(num_classes=18)
train_variable_length_model(model, train_loader, val_loader)

/Users/dhanush/miniconda3/envs/musaic_env/lib/python3.9/site-packages/madmom/io/audio.py:643: WavFileWarning: Chunk (non-data) not understood, skipping it.
  file_sample_rate, signal = wavfile.read(filename, mmap=True)


0 out of 404 training complete


/Users/dhanush/miniconda3/envs/musaic_env/lib/python3.9/site-packages/madmom/io/audio.py:643: WavFileWarning: Chunk (non-data) not understood, skipping it.
  file_sample_rate, signal = wavfile.read(filename, mmap=True)


1 out of 404 training complete


/Users/dhanush/miniconda3/envs/musaic_env/lib/python3.9/site-packages/madmom/io/audio.py:643: WavFileWarning: Chunk (non-data) not understood, skipping it.
  file_sample_rate, signal = wavfile.read(filename, mmap=True)


...

98 out of 101 validation complete


/Users/dhanush/miniconda3/envs/musaic_env/lib/python3.9/site-packages/madmom/io/audio.py:643: WavFileWarning: Chunk (non-data) not understood, skipping it.
  file_sample_rate, signal = wavfile.read(filename, mmap=True)


99 out of 101 validation complete


/Users/dhanush/miniconda3/envs/musaic_env/lib/python3.9/site-packages/madmom/io/audio.py:643: WavFileWarning: Chunk (non-data) not understood, skipping it.
  file_sample_rate, signal = wavfile.read(filename, mmap=True)


100 out of 101 validation complete
Epoch 10/10:
  Train Loss: 1.9374, Acc: 35.64%
  Val Loss: 2.3609, Acc: 34.65%


In [33]:
PATH = "trumpet_cnn_v1.pt"
torch.save(model.state_dict(), PATH)